In [1]:
# scraping_restaurantes.ipynb - Celda 1
import sys
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configurar paths para importar nuestros módulos
sys.path.append(os.getcwd())
from src.utils.database import get_db_connection, insert_dataframe_to_table

print("✅ Módulos importados correctamente")

✅ Módulos importados correctamente


In [3]:
# scraping_restaurantes.ipynb - Celda 2
def get_driver():
    """Configura y devuelve un driver de Selenium con opciones para evitar detección"""
    options = uc.ChromeOptions()
    # options.add_argument('--headless')  # Descomenta si quieres que corra en segundo plano
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--start-maximized')
    
    driver = uc.Chrome(options=options)
    return driver

# Probamos el driver
driver = get_driver()
driver.get("https://www.tripadvisor.es/")
print("✅ Navegador configurado correctamente")
time.sleep(3)
driver.quit()  # Cerramos esta instancia de prueba

✅ Navegador configurado correctamente


In [5]:
# Celda de diagnóstico
driver = get_driver()
driver.get("https://www.tripadvisor.es/Restaurants-g187499-Girona_Province_of_Girona_Catalonia.html")

# Espera un poco y luego guarda el HTML de la página para inspeccionarlo
time.sleep(5)
html_content = driver.page_source

# Guarda el HTML en un archivo para analizarlo
with open("pagina_tripadvisor.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("✅ HTML guardado en 'pagina_tripadvisor.html'")
driver.quit()

# Ahora ABRE ese archivo en tu editor de código y revisa:
# 1. ¿Aparece el contenido de restaurantes?
# 2. ¿Qué selectores CSS puedes usar?

✅ HTML guardado en 'pagina_tripadvisor.html'


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import sqlite3
import time
import re

# --- Configuración de la Base de Datos ---
conn = sqlite3.connect('restaurantes.db')
cursor = conn.cursor()

# --- Configuración de Selenium ---
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)  # Aumentamos el tiempo de espera

offsets = [120, 150, 180, 210]

def aceptar_cookies(driver):
    try:
        boton_accept = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        boton_accept.click()
        print("   ✅ Cookies aceptadas")
        time.sleep(1)
        return True
    except TimeoutException:
        print("   ℹ️ No apareció el banner de cookies")
        return False

def hacer_scroll_hasta_abajo(driver):
    """Función para hacer scroll hasta el final de la página"""
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Espera a que cargue el contenido después del scroll

try:
    for offset in offsets:
        print(f"Scrapeando página con offset {offset}...")
        url = f"https://www.tripadvisor.es/Restaurants-g187499-oa{offset}-Girona_Province_of_Girona_Catalonia.html"
        driver.get(url)
        
        aceptar_cookies(driver)
        time.sleep(3)
        
        # Hacer scroll para asegurar que se cargan todos los elementos
        hacer_scroll_hasta_abajo(driver)

        # PRUEBA DIFERENTES SELECTORES HASTA QUE FUNCIONE UNO
        restaurantes = []
        selectores_a_probar = [
            "[data-test*='restaurant']",
            "[data-test*='venue']",
            ".listing", 
            ".listItem",
            ".prw_rup",
            ".ui_column",
            ".restaurant"
        ]
        
        for selector in selectores_a_probar:
            try:
                restaurantes = driver.find_elements(By.CSS_SELECTOR, selector)
                if len(restaurantes) > 0:
                    print(f"   ✅ Selector encontrado: '{selector}' con {len(restaurantes)} elementos")
                    break
            except:
                continue
        
        if not restaurantes:
            print(f"   ❗ No se encontraron restaurantes con ningún selector. Tomando screenshot...")
            driver.save_screenshot(f"error_offset_{offset}.png")
            continue

        print(f"   Procesando {len(restaurantes)} restaurantes...")

        for resto in restaurantes:
            try:
                # DIFERENTES SELECTORES PARA EL NOMBRE
                nombre_completo_element = None
                selectores_nombre = [
                    "a[href*='/Restaurant_Review']",
                    ".restaurantName",
                    ".wQjYi",
                    "a[class*='restaurant']"
                ]
                
                for selector_n in selectores_nombre:
                    try:
                        nombre_completo_element = resto.find_element(By.CSS_SELECTOR, selector_n)
                        if nombre_completo_element:
                            break
                    except:
                        continue
                
                if not nombre_completo_element:
                    print("   ❗ No se pudo encontrar el nombre del restaurante")
                    continue
                    
                texto_completo = nombre_completo_element.text.strip()
                print(f"Texto completo: {texto_completo}")

                # Separar número y nombre
                match = re.match(r'^(\d+)\.\s*(.+)$', texto_completo)
                if match:
                    posicion_ranking = int(match.group(1))
                    nombre = match.group(2)
                    if not (120 <= posicion_ranking <= 239):
                        continue
                else:
                    print(f"⚠️ No se pudo separar el ranking: '{texto_completo}'")
                    posicion_ranking = None
                    nombre = texto_completo
                
                # Resto del código para puntuación, cocina, precio...
                # (Mantén el mismo código de antes para estas partes)

                print(f"{posicion_ranking}. {nombre}")
                # ... [aquí iría el resto del procesamiento]

            except Exception as e:
                print(f"❌ Error procesando restaurante: {e}")
                continue

        time.sleep(2)

finally:
    conn.commit()
    cursor.close()
    conn.close()
    driver.quit()
    print("Scraping finalizado.")

Scrapeando página con offset 120...
   ✅ Cookies aceptadas
   ❗ No se encontraron restaurantes con ningún selector. Tomando screenshot...
Scrapeando página con offset 150...
   ℹ️ No apareció el banner de cookies
   ❗ No se encontraron restaurantes con ningún selector. Tomando screenshot...
Scrapeando página con offset 180...
   ℹ️ No apareció el banner de cookies
   ❗ No se encontraron restaurantes con ningún selector. Tomando screenshot...
Scrapeando página con offset 210...
   ℹ️ No apareció el banner de cookies
   ❗ No se encontraron restaurantes con ningún selector. Tomando screenshot...
Scraping finalizado.
